# Project 5: Working with Pandas and SQL Databases (Movies Dataset)

# Project Brief for Self-Coders

Here you´ll have the opportunity to code major parts of Project 5 on your own. If you need any help or inspiration, have a look at the Videos or the Jupyter Notebook with the full code. <br> <br>
Keep in mind that it´s all about __getting the right results/conclusions__. It´s not about finding the identical code. Things can be coded in many different ways. Even if you come to the same conclusions, it´s very unlikely that we have the very same code. 

## Creating an SQLite Database

1. __Import__ sqlite3 (as sq3) and __create__ a new SQLite Database with the name __"movies.db"__.

In [83]:
import sqlite3 as sq3

In [104]:
con = sq3.connect("movies.db")

In [85]:
con

In [86]:
con.execute("SELECT * FROM sqlite_master").fetchall()

[]

In [87]:
con.close()

## Loading Data from DataFrames into an SQLite Database

2. __Load__ the json file __"some_movies.json"__ and __split__ the dataset into the following __four datasets__ (save each dataset in a Pandas DataFrame).

__Dataset #1 (Movies)__ with columns ["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]. <br>
Convert "release_date" to datetime and transform "budget" and "revenue" to Million USD before loading into the Database. 

__Dataset #2 (Votes)__ with columns ["id", "vote_count", "vote_average"]. 

__Dataset #3 (Genres)__ with columns ["genre_id", "genre_name", "id"]. <br> 

__Dataset #4 (Prod)__ with columns ["comp_id", "comp_logo_path", "comp_name", "comp_origin_country", "id" ]. <br>


In [88]:
import pandas as pd
import json

In [89]:
with open("some_movies.json") as f:
    data = json.load(f)

In [90]:
df = pd.json_normalize(data, sep = "_")

In [91]:
df.shape

(18, 29)

In [92]:
df.columns

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'belongs_to_collection'],
      dtype='object')

In [93]:
df.release_date = pd.to_datetime(df.release_date)

In [94]:
def get_genresid(genres):
    for i in genres:
        return i['id']

In [120]:
def get_genrename(genres):
    for i in genres:
        return i['name']

In [14]:
df["genre_id"] = df.genres.apply(lambda x: get_genresid(x) if isinstance(x,list) else np.nan)

In [15]:
df.genre_id.value_counts(dropna = False)

28       8
12       3
16       2
878      2
14       1
18       1
10751    1
Name: genre_id, dtype: int64

In [122]:
df.genres.apply(lambda x: "|".join(i['name'] for i in x) )

0              Adventure|Science Fiction|Action
1      Action|Adventure|Fantasy|Science Fiction
2      Action|Adventure|Science Fiction|Fantasy
3              Adventure|Action|Science Fiction
4                                 Drama|Romance
5     Action|Adventure|Science Fiction|Thriller
6                              Adventure|Family
7              Science Fiction|Action|Adventure
8                               Action|Thriller
9              Action|Adventure|Science Fiction
10     Action|Adventure|Fantasy|Science Fiction
11                            Fantasy|Adventure
12             Science Fiction|Action|Adventure
13                   Animation|Family|Adventure
14             Action|Adventure|Science Fiction
15                   Animation|Adventure|Family
16                       Family|Fantasy|Romance
17            Action|Adventure|Animation|Family
Name: genres, dtype: object

In [96]:
movies = df[["id", "title", "revenue", "budget", "belongs_to_collection_name", "release_date"]].copy()
votes = df[["id", "vote_count", "vote_average"]].copy()
#genres =df[["genre_id", "genre_name", "id"]].copy()
genres = pd.json_normalize(data = data, record_path = "genres", meta = "id", record_prefix = "genre_")
prod = pd.json_normalize(data = data, record_path = "production_companies", meta = "id",record_prefix = "comp_")

In [114]:
data[0]['genres']

[{'id': 12, 'name': 'Adventure'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 28, 'name': 'Action'}]

In [116]:
movies.shape
genres.shape

(55, 3)

In [97]:
movies.revenue = df.revenue/1000000
movies.budget = df.budget/1000000

3. __Load__ the datasets __into the database__ (each dataset should be a separate table in the database). __Name__ the tables "Movies", "Votes", "Genres", "Prod".

In [98]:
con = sq3.connect("movies.db")

In [99]:
movies.to_sql("Movies",con, index = False)
votes.to_sql("Votes", con, index  = False)
genres.to_sql("Genres", con, index = False)
prod.to_sql("Prod",con, index = False)

In [100]:
con.execute("SELECT * FROM sqlite_master").fetchall()

[('table',
  'Movies',
  'Movies',
  2,
  'CREATE TABLE "Movies" (\n"id" INTEGER,\n  "title" TEXT,\n  "revenue" REAL,\n  "budget" REAL,\n  "belongs_to_collection_name" TEXT,\n  "release_date" TIMESTAMP\n)'),
 ('table',
  'Votes',
  'Votes',
  3,
  'CREATE TABLE "Votes" (\n"id" INTEGER,\n  "vote_count" INTEGER,\n  "vote_average" REAL\n)'),
 ('table',
  'Genres',
  'Genres',
  4,
  'CREATE TABLE "Genres" (\n"genre_id" INTEGER,\n  "genre_name" TEXT,\n  "id" INTEGER\n)'),
 ('table',
  'Prod',
  'Prod',
  5,
  'CREATE TABLE "Prod" (\n"comp_id" INTEGER,\n  "comp_logo_path" TEXT,\n  "comp_name" TEXT,\n  "comp_origin_country" TEXT,\n  "id" INTEGER\n)')]

In [101]:
con.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name").fetchall()

[('Genres',), ('Movies',), ('Prod',), ('Votes',)]

In [102]:
con.close()

## Loading Data from SQLite Databases into DataFrames

4. __Load__ the full tables "Movies", "Votes", "Genres", "Prod" from "movies.db" into Pandas (four DataFrames). __Set__ "id" as Index. 

In [24]:
con = sq3.connect("movies.db")

In [25]:
pd.read_sql("SELECT * FROM Movies",con, index_col = 'id', parse_dates= 'release_date')

,title,revenue,budget,belongs_to_collection_name,release_date
id,,,,,
299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24
19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10
140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15
299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25
597,Titanic,1845.034188,200.0,None,1997-11-18
135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06
420818,The Lion King,1656.943394,260.0,None,2019-07-12
24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25
168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01


In [26]:
con.close()

##  Some Simple SQL Queries

5. __Perform__ the following simple __SQL Queries__ and __store__ the results in DataFrames:

In [27]:
con =sq3.connect("movies.db")

__Load the full "Movies" Table__.

In [28]:
pd.read_sql("SELECT * FROM Movies",con, index_col='id', parse_dates='release_date')

,title,revenue,budget,belongs_to_collection_name,release_date
id,,,,,
299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24
19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10
140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15
299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25
597,Titanic,1845.034188,200.0,None,1997-11-18
135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06
420818,The Lion King,1656.943394,260.0,None,2019-07-12
24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25
168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01


__Load the columns "id", "revenue" and "release_date" from "Movies".__ 

In [ ]:
pd.read_sql("SELECT id,revenue, release_date FROM Movies",con)

__Get the Total Revenue (sum) over all movies from "Movies".__

In [ ]:
"SELECT sum(revenue) FROM Movies)"

__Count the number of Movies in "Movies".__

In [29]:
pd.read_sql("SELECT count(title) FROM Movies", con)

,count(title)
0,18


__Count the number of Movies that do belong to a collection.__

In [ ]:
pd.read_sql('SELECT count(title) FROM Movies WHERE belongs_to_collection_name != "None"',con)

__Get the average budget from "Movies"__.

In [ ]:
pd.read_sql('SELECT avg(budget) FROM Movies', con)

## Some more SQL Queries

6. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Load all columns for the movie with movie id 597__.

In [30]:
pd.read_sql("SELECT * FROM Movies WHERE id = 597", con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,597,Titanic,1845.034188,200.0,None,1997-11-18 00:00:00


__Load all columns for all movies with a revenue greater than 2000 (MUSD).__

In [32]:
pd.read_sql("SELECT * FROM Movies WHERE revenue > 2000", con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25 00:00:00


__Load all columns for all movies with a revenue greater than 1500 (MUSD) and a budget below 200 (MUSD).__

In [34]:
pd.read_sql("SELECT * FROM Movies WHERE revenue >1500 AND budget < 200", con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06 00:00:00
1,168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01 00:00:00


__Get the minimum budget from those movies with a revenue greater than 1250 (MUSD).__

In [41]:
pd.read_sql("SELECT MIN(budget) FROM Movies WHERE revenue > 1250", con)

,MIN(budget)
0,125.0


__Get all unique collection Names from "Movies".__

In [43]:
pd.read_sql("SELECT DISTINCT belongs_to_collection_name FROM Movies",con)

,belongs_to_collection_name
0,The Avengers Collection
1,Avatar Collection
2,Star Wars Collection
3,None
4,Jurassic Park Collection
5,The Fast and the Furious Collection
6,Black Panther Collection
7,Harry Potter Collection
8,Frozen Collection
9,The Incredibles Collection


__Load all movies (all columns) and sort by budget from high to low.__

In [44]:
pd.read_sql("SELECT * FROM Movies ORDER BY budget DESC",con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00
1,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25 00:00:00
2,420818,The Lion King,1656.943394,260.0,None,2019-07-12 00:00:00
3,99861,Avengers: Age of Ultron,1405.403694,250.0,The Avengers Collection,2015-04-22 00:00:00
4,140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15 00:00:00
5,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00
6,24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25 00:00:00
7,597,Titanic,1845.034188,200.0,None,1997-11-18 00:00:00
8,284054,Black Panther,1346.739107,200.0,Black Panther Collection,2018-02-13 00:00:00
9,181808,Star Wars: The Last Jedi,1332.539889,200.0,Star Wars Collection,2017-12-13 00:00:00


__Load all movies (all columns) that do not belong to a collection.__

In [56]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name IS NULL",con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,597,Titanic,1845.034188,200.0,None,1997-11-18 00:00:00
1,420818,The Lion King,1656.943394,260.0,None,2019-07-12 00:00:00
2,321612,Beauty and the Beast,1263.521126,160.0,None,2017-03-16 00:00:00


__Load all movies (all columns) that belong to a collection.__

In [57]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name IS NOT NULL",con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00
1,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00
2,140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15 00:00:00
3,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25 00:00:00
4,135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06 00:00:00
5,24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25 00:00:00
6,168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01 00:00:00
7,99861,Avengers: Age of Ultron,1405.403694,250.0,The Avengers Collection,2015-04-22 00:00:00
8,284054,Black Panther,1346.739107,200.0,Black Panther Collection,2018-02-13 00:00:00
9,12445,Harry Potter and the Deathly Hallows: Part 2,1341.511219,125.0,Harry Potter Collection,2011-07-07 00:00:00


__Load all movies (all columns) where "Avengers..." is in the title__.

In [59]:
pd.read_sql("SELECT * FROM Movies WHERE title LIKE '%Avenger%'",con)

,id,title,revenue,budget,belongs_to_collection_name,release_date
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00
1,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25 00:00:00
2,24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25 00:00:00
3,99861,Avengers: Age of Ultron,1405.403694,250.0,The Avengers Collection,2015-04-22 00:00:00


## Join Queries

7. __Perform__ the following __SQL Join Queries__ and __store__ the results in DataFrames:

__Join "Movies" and "Votes"__ (all columns).

In [60]:
pd.read_sql("SELECT * \
             FROM Movies m JOIN Votes v\
             ON m.id = v.id",con)

,id,title,revenue,budget,belongs_to_collection_name,release_date,id,vote_count,vote_average
0,299534,Avengers: Endgame,2797.800564,356.0,The Avengers Collection,2019-04-24 00:00:00,299534,12607,8.3
1,19995,Avatar,2787.965087,237.0,Avatar Collection,2009-12-10 00:00:00,19995,21000,7.4
2,140607,Star Wars: The Force Awakens,2068.223624,245.0,Star Wars Collection,2015-12-15 00:00:00,140607,14205,7.4
3,299536,Avengers: Infinity War,2046.239637,300.0,The Avengers Collection,2018-04-25 00:00:00,299536,17718,8.3
4,597,Titanic,1845.034188,200.0,None,1997-11-18 00:00:00,597,16661,7.8
5,135397,Jurassic World,1671.713208,150.0,Jurassic Park Collection,2015-06-06 00:00:00,135397,15399,6.6
6,420818,The Lion King,1656.943394,260.0,None,2019-07-12 00:00:00,420818,5425,7.2
7,24428,The Avengers,1519.557910,220.0,The Avengers Collection,2012-04-25 00:00:00,24428,22101,7.7
8,168259,Furious 7,1515.047671,190.0,The Fast and the Furious Collection,2015-04-01 00:00:00,168259,7359,7.3
9,99861,Avengers: Age of Ultron,1405.403694,250.0,The Avengers Collection,2015-04-22 00:00:00,99861,15548,7.3


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average").__

In [61]:
pd.read_sql("SELECT m.id, m.title, v.vote_average \
             FROM Movies m JOIN Votes v\
             ON m.id = v.id",con)

,id,title,vote_average
0,299534,Avengers: Endgame,8.3
1,19995,Avatar,7.4
2,140607,Star Wars: The Force Awakens,7.4
3,299536,Avengers: Infinity War,8.3
4,597,Titanic,7.8
5,135397,Jurassic World,6.6
6,420818,The Lion King,7.2
7,24428,The Avengers,7.7
8,168259,Furious 7,7.3
9,99861,Avengers: Age of Ultron,7.3


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8.__

In [62]:
pd.read_sql("SELECT m.id, m.title, v.vote_average \
             FROM Movies m JOIN Votes v\
             ON m.id = v.id\
             WHERE v.vote_average > 8",con)

,id,title,vote_average
0,299534,Avengers: Endgame,8.3
1,299536,Avengers: Infinity War,8.3
2,12445,Harry Potter and the Deathly Hallows: Part 2,8.1


__Join "Movies" and "Votes" (only the columns "id", "title", "vote_average") and return only those movies with vote_average > 8 and in ascending budget order__.

In [64]:
pd.read_sql("SELECT m.id, m.title, v.vote_average \
             FROM Movies m JOIN Votes v\
             ON m.id = v.id\
             WHERE v.vote_average > 8\
             ORDER BY m.budget",con)

,id,title,vote_average
0,12445,Harry Potter and the Deathly Hallows: Part 2,8.1
1,299536,Avengers: Infinity War,8.3
2,299534,Avengers: Endgame,8.3


## Final Case Study

8. __Perform__ the following advanced __SQL Queries__ and __store__ the results in DataFrames:

__Get the Total Revenue (sum) for each Production Company.__

In [105]:
pd.read_sql("SELECT p.comp_name, SUM(m.revenue) as rev \
             FROM Movies m JOIN Prod p\
             ON m.id = p.id\
             GROUP BY 1\
             ORDER BY rev DESC",con)

,comp_name,rev
0,Walt Disney Pictures,9446.618940
1,Marvel Studios,9115.740912
2,20th Century Fox,4632.999275
3,Lightstorm Entertainment,4632.999275
4,Universal Pictures,4490.220464
5,Lucasfilm,3400.763513
6,Paramount,3364.592098
7,Dentsu,3186.760879
8,Fuji Television Network,3186.760879
9,Amblin Entertainment,2975.172793


__Get all Production Companies for the movie "Titanic".__

In [106]:
pd.read_sql("SELECT p.comp_name \
             FROM Movies m JOIN Prod p\
             ON m.id = p.id\
             WHERE m.title = 'Titanic'",con)

,comp_name
0,20th Century Fox
1,Lightstorm Entertainment
2,Paramount


__Get the Total Revenue (sum) for each Genre.__

In [107]:
pd.read_sql("SELECT g.genre_name, SUM(m.revenue) as rev \
             FROM Movies m JOIN Genres g\
             ON m.id = g.id\
             GROUP BY 1\
             ORDER BY rev DESC",con)

,genre_name,rev
0,Adventure,25124.972342
1,Action,21036.581432
2,Science Fiction,18279.642305
3,Fantasy,8807.960163
4,Family,6767.339944
5,Animation,3846.875424
6,Thriller,3186.760879
7,Romance,3108.555314
8,Drama,1845.034188


__Get all Genres for the movie "Frozen II".__

In [108]:
pd.read_sql("SELECT Genres.id, Genres.genre_name, Movies.revenue, Movies.title \
            FROM Genres \
            LEFT JOIN Movies \
            ON Genres.id=Movies.id", con)

,id,genre_name,revenue,title
0,299534,Adventure,2797.800564,Avengers: Endgame
1,299534,Science Fiction,2797.800564,Avengers: Endgame
2,299534,Action,2797.800564,Avengers: Endgame
3,19995,Action,2787.965087,Avatar
4,19995,Adventure,2787.965087,Avatar
5,19995,Fantasy,2787.965087,Avatar
6,19995,Science Fiction,2787.965087,Avatar
7,140607,Action,2068.223624,Star Wars: The Force Awakens
8,140607,Adventure,2068.223624,Star Wars: The Force Awakens
9,140607,Science Fiction,2068.223624,Star Wars: The Force Awakens


In [109]:
pd.read_sql("SELECT Genres.genre_name \
            FROM Genres \
            LEFT JOIN Movies \
            ON Genres.id=Movies.id \
            WHERE Movies.title = 'Frozen II'", con)

,genre_name
0,Adventure
1,Animation
2,Family


# +++++++++ See some Hints below +++++++++++++

# ++++++++++++++++ Hints++++++++++++++++++++

__Hints for 1.__<br>
You can do this with sq3.connect("database_name.db")

__Hints for 2.__ <br>
You have to use pd.json_normalize(data = ..., record_path = ..., meta = ..., record_prefix = ... ) for Datasets #3 and #4 

__Hints for 3.__<br>
You can do this with: 

In [ ]:
con = sq3.connect("movies.db")
df.to_sql("Table Name", con, index = False)

__Hints for 4.__<br>
You can do this with:

In [ ]:
con = sq3.connect("movies.db")
pd.read_sql("SELECT * FROM Table Name", con, index_col = ...)

__Hints for 5., 6., 7., 8.__<br>
You can do this with:

In [ ]:
con = sq3.connect("movies.db")
df = pd.read_sql("insert the sql query here", con)